# Autenticidad, 
# los Mensajes Secretos,
# y Seguridad



<a href="https://colab.research.google.com/github/ProfDoeg/Colegio_Invisible/blob/master/04_cuaderno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparar el Espacio (Prepare the Space)

In [2]:
! pip install eciespy eth_keys gspread oauth2client
! git clone https://github.com/ProfDoeg/Colegio_Invisible.git
! pip install --upgrade gspread

     |████████████████████████████████| 555 kB 21.1 MB/s 
     |████████████████████████████████| 2.0 MB 48.8 MB/s 
     |████████████████████████████████| 104 kB 51.3 MB/s 
     |████████████████████████████████| 481 kB 51.0 MB/s 
  Created wheel for cytoolz: filename=cytoolz-0.11.2-cp37-cp37m-linux_x86_64.whl size=1230789 sha256=f11a5fe107f228e7072416192b847130ae92331eeed6e999b7816694db3f4ad8
  Stored in directory: /root/.cache/pip/wheels/38/70/71/ca13ea3d36ccd0b3d0ec7d7a4ca67522048d695b556bba4f59
Successfully built cytoolz
Cloning into 'Colegio_Invisible'...
remote: Enumerating objects: 648, done.
remote: Counting objects: 100% (462/462), done.
remote: Compressing objects: 100% (437/437), done.
remote: Total 648 (delta 317), reused 23 (delta 23), pack-reused 186
Receiving objects: 100% (648/648), 42.57 MiB | 4.60 MiB/s, done.
Resolving deltas: 100% (429/429), done.
  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Su

# Historias

<img src="https://github.com/ProfDoeg/Colegio_Invisible/raw/main/img/barb_2.jpeg" width="250"/>


*¡Verdezuela, Verdezuela,\
Suéltame tu cabellera!*\
-La Bruja

*Encryption is a powerful defensive\
weapon for free people.*\
-Esther Dyson




# Aplicar Criptografía Para Crear un Ensamblaje de Mensajería Seguro 
# (Apply Cryptography to Create a Secure Messaging Assembly)

## Esquema (Outline) 

Aplicar criptografía para crear un tablero de mensajes securos.
***
Apply cryptography to build a secure messaging board.

## Metas (Goals)

- crear y guardar claves\
(create and save keys)
- descargar/cargar claves\
(download/upload keys)
- descifrar el token de acceso a la hoja de cálculo\
(decrypt spreadsheet access token)
- entender `gspread`\
(understand `gspread`)
- publicar: nombre, clave pública, mensaje público y firma\
(post: name, public key, public message, and signature)
- firmar mensaje\
(sign message)
- verificar mensaje\
(verify message)
- escribir mensaje a otros participantes\
(write message to other participants)
- escribir un mensaje encriptado a otro participante\
(write encrypted message to other participant)
- escribir mensaje firmado y encriptado\
(write signed and encrypted message)


# Gestionar Archivos de Claves (Manage Key Files)

## Obtener Claves (Get Keys)

3 Opciones (3 Options):
- hacer nuevas claves\
(make new keys)
- cargar claves generadas previamente\
(load previously generated keys)
  - desde la disco local\
  (from local drive)
  - desde Google Drive\
  (from Google Drive)


### Hacer Nuevas Claves (Make New Keys)

In [6]:
! mkdir llaves

In [7]:
! python Colegio_Invisible/scripts/ecc_generate.py llaves/prvkey1.eck

Input password for encrypting keyfile:
Ingresar password para cifrar el archivo de la clave:
Repetir (Repeat): 

Passwords match...

Passwords son iguales...
Generación de la clave privada está terminada (Private key generation complete)
La clave privada está cifrada y escrita en un archivo binario:
Private key encrypted and written to binary file: llaves/prvkey1.eck
66cb3828d13377934b38fa235fe8d49ae6d38b88a0016e8c62f6d41b2ffd342e2c605c0903ebedb7ed8382b01ddd8ef947603e87a98d2aa548744bba19e7b29c


In [8]:
! python Colegio_Invisible/scripts/ecc_pubkey_extract.py llaves/prvkey1.eck llaves/pubkey1.eck 

Input password for private key file:
Ingresar password para acceder al archivo del la clave privada:
Input password for public key file:
Ingresar password para acceder al archivo del la clave pública: 
Repetir (Repeat): 

Passwords match...

Passwords son iguales...
Extracción de la clave pública terminada desde
(Completed public key extraction from): llaves/prvkey1.eck
La clave pública está cifrada y escrita en archivo binario
(Public key encrypted and written to binary file): llaves/pubkey1.eck
3b5a8c15885fae604cb40ed3aafb14610b173562bd3b06ee7fef09b103523eb38ae4cdaf2bef3a6a514d93c82d685e8cba99c6743c7d2bab155c3d07e94fae0a6d1f77ee26717fc6c3aaa86c672c389ccd0f61ada21f324413b6316348b897b1


### Cargar Claves (Load Keys)

**EJECUTAR SOLO SI HAS GUARDADO PREVIAMENTE LAS CLAVES**
***
**ONLY RUN IF YOU HAVE PREVIOUSLY SAVED KEYS**

#### De Disco Local (From Local Disk)

In [ ]:
from google.colab import files
fu=files.upload();

In [ ]:
! unzip llaves.zip

#### De Google Drive (From Google Drive)

1. Montar Google Drive\
(Mount Google Drive)
2. Copiar el archivo de clave desde la ubicación al directorio actual\
(Copy key file from location to present directory)
3. Unzip archivo de clave\
(Unzip key file)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! cp drive/MyDrive/cinv/llaves.zip llaves.zip 
! unzip llaves.zip

### Guardar Llaves (Save Keys) 

#### A Disco Local (To Local Disk)

In [ ]:
#note
! zip -r llaves.zip llaves

In [ ]:
#note
from google.colab import files
f=files.download('llaves.zip');

#### A Google Drive (To Google Drive)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! cp llaves.zip drive/MyDrive/cinv/llaves.zip 

# Usar Guion Para Acceder Clave Pública (Use Script to Access Public Key)

In [9]:
ret=!{ 'python Colegio_Invisible/scripts/ecc_keydump.py llaves/pubkey1.eck'}
pubkey1=ret[-1]
pubkey1

'8a58c1e7a54a9f48b21e57d2600ca2a35659d6c488e7ef7c5b6280edee632857fb8c233b294e24f516fd4b102d4b9cbfbc61c2566ce8f57db0ed6d05e1de7bc9'

# Gestionar el Acceso a Google Sheets (Manage Access to Google Sheets)


## Descifrar Credenciales de API (Decrypt API Credentials)

$\color{white}{\text{c0leg101nv1s1ble}}$

Usar la contraseña de arriba para descifrar `client_secret.aes`
***
Use the above password to decrypt `client_secret.aes`





In [11]:
! python Colegio_Invisible/scripts/aes_decrypt.py Colegio_Invisible/secrets/client_secret.aes Colegio_Invisible/secrets/client_secret.json 

Input password for decryption:
Ingresar password para decifrar: 
Éxito (Success): Colegio_Invisible/secrets/client_secret.aes decifrado con éxito (decryption complete)
Escrito a (Written to): Colegio_Invisible/secrets/client_secret.json


In [12]:
! ls Colegio_Invisible/secrets

client_secret.aes  client_secret.json


## `gspread` 🪴



`gspread` es un módulo de Python para la interacción programática con Google Sheets.\
Todos los participantes tendrán acceso a la misma hoja de cálculo en la nube.
***
`gspread` is a python module for programmatic interaction with Google Sheets.\
All the participants will have access to the same spreadsheet in the cloud.



https://docs.gspread.org/en/latest/user-guide.html

Usar las credenciales descifradas para autenticar y obtener acceso API a la hoja de cálculo
***
Use the decrypted credentials to authenticate and gain API access to the spreadsheet

In [12]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('Colegio_Invisible/secrets/client_secret.json', scope)
client = gspread.authorize(creds)
sheets = client.open("signatures")

# Sheets

Aquí podemos ver la estructura de la hoja de cálculo y todas las capas o `Worksheet`
***
Here we can see the structure of the spreadsheet and all of the layers or `Worksheet`

In [13]:
sheets.worksheets()

[<Worksheet 'Multi' id:0>,
 <Worksheet 'Magic' id:2092809244>,
 <Worksheet 'ChorusSignature' id:549527758>,
 <Worksheet 'StealProposal' id:858983463>,
 <Worksheet 'StealSignature' id:475665302>,
 <Worksheet 'test' id:2104752728>]

In [14]:
sheet=sheets.worksheets()[0]
sheet1=sheets.worksheets()[1]
sheet2=sheets.worksheets()[2]
sheet3=sheets.worksheets()[3]
sheet,sheet1,sheet2,sheet3

(<Worksheet 'Multi' id:0>,
 <Worksheet 'Magic' id:2092809244>,
 <Worksheet 'ChorusSignature' id:549527758>,
 <Worksheet 'StealProposal' id:858983463>)

## Sheet 0 : Registro (Roster)

Aislar la primera hoja donde consultamos la primera fila
***
Isolate the first sheet where we query the first row

In [15]:
sheet.row_values(1)

['name', 'pubkey', 'statement', 'signature']

**PELIGRO: NO EJECUTAR**
***
**DANGER: DO NOT RUN**

In [16]:
#sheet.clear()
#sheet.update_cell(1, "name")
#sheet.append_row(["name", "pubkey","statement","signature"])
#sheet.__dir__()

### Tomar el Registro (Make the Roster)

Publicar (Publish):

- `name` nombre de participante\
(`name` title of participant)
- `pubkey` clave público de participante en hexadecimal\
(`pubkey` public key of participant in hex)
- `statement` la declaración firmada por el participante\
(`statement` data signed by participant)
- `signature` firma ecc de la declaración\
(`signature` ecc signature of the statement)

#### Entrada Datos (Input Data)

In [29]:
my_name=input('input name')
my_statement=input('input statement')
open('llaves/statement1.txt','wb').write(my_statement.encode())

input namejj
input statementyo jj


5

In [30]:
! cat llaves/statement1.txt

yo jj

Usar script para firmar la declaración
***
Use script to sign statement

In [31]:
ret = !{ 'python Colegio_Invisible/scripts/ecc_sign.py llaves/prvkey1.eck llaves/statement1.txt llaves/sig1.sig'}
sig1=ret[-1]
sig1

'118a2aae4484ef5742913ffbde97981438fb54ba22ed564dba8f108e656e7aa1114e47a8efeca893885bec9e4ea0793645b810895a9f385d14d62805e2ac992101'

#### Publicar Datos (Publish Data)


In [33]:
peeps=sheet.col_values(1)[1:]
datas=[my_name,pubkey1,my_statement,sig1]
if not (my_name in peeps) :
  sheet.append_row(datas)
else:
  row=peeps.index(my_name)+2
  for i,value in enumerate(datas[1:]):
    sheet.update_cell(row=row,col=i+2,value=value)

## Leer Sheet 0 (Read Sheet 0)

In [50]:
import pandas as pd

dataframe = pd.DataFrame(sheet.get_all_records())
dataframe

,name,pubkey,statement,signature
0,DrDoeg,aa403c5cc3bd3f8970f4b69a19b19f758a01b8b7df51dc...,I am DrDoeg,2671da66309b34b396f42983350e213205f0510124058d...
1,lil late,d4870b35f981df8a87545c0d4e3879c52417e3c914cbf4...,das aright,2c10c648ea437d08e33f0a28a83c5ca56986d23b3e046f...
2,Paulina,f7ee21788227541ac7d9d684fed40f1c89772ebba67027...,I am Paulina R,346d7e2fad7856a8b86ef1435fb328284ef123668c39a2...
3,Lau,cbf0666e1f02013145d325d813aaaa5f0ffd49361136cf...,Soy Lau,7eb257548a943314a2818a1a9f649c6cfc1d9c475e598b...
4,Pamela,74bb3d46084d27eb4341b5e3531b16e8911bb6fb765aa9...,Soy Pamela,f68597821580b15584f42574b812d3d3b4b043ffb70dbe...
5,Seco,7da1b363b5942d7aeeeedb092520701b44f12e39d6c297...,I am Seco,Fallada (Fail) [Errno 2] No such file or direc...
6,Joeq,8081496f8bdca4dac2ffa446df3901b6b020ef621fa1c0...,Joeq Nince,a5fcf6f0589868bb118b9938936bd3b600a31159699426...
7,jj,8a58c1e7a54a9f48b21e57d2600ca2a35659d6c488e7ef...,yo jj,118a2aae4484ef5742913ffbde97981438fb54ba22ed56...


## Verificar Sheet0 Firma/Declaración/Clave Pública (Verify Sheet0 Signature/Statement/PublicKey)

Usar la biblioteca de criptografía `ecies` directamente sin usar los scripts externos.
***
Use the `ecies` cryptography library directly without using the external scripts.

- importar la biblioteca\
 (import library)
- definir la función de verificación\
(define verification function)
- probar la función\
(test function)
- aplicar a cada fila de la tabla\
(apply to every row of table)

In [46]:
import eth_keys
import ecies

def verify(sig,statement,pubkey):
  try:
    return eth_keys.datatypes.Signature(bytes.fromhex(sig)).verify_msg(statement.encode() if type(statement)!=bytes else statement,eth_keys.keys.PublicKey(bytes.fromhex(pubkey)))
  except Exception as E:
    return E
verify(sig1,my_statement,pubkey1)

True

In [47]:
dataframe.set_index('name').apply(lambda row: verify(row.signature,row.statement,row.pubkey),axis=1)

name
DrDoeg                                                   True
lil late                                                 True
Paulina                                                  True
Lau                                                      True
Pamela                                                   True
Seco        non-hexadecimal number found in fromhex() arg ...
Joeq                                                     True
jj                                                       True
dtype: object

# Configurar Tableros de Comunicación: Sheets 1-3
# (Setup Communication Boards: Sheets 1-3)


In [54]:
#ONLY FOR DDOG MC
hojas=[sheet1,sheet2,sheet3]
participants = dataframe.name.tolist()
df=pd.DataFrame('None',columns=participants,index=participants)

def fill_sheets(h,dfrm):
  for hoja in h:
    hoja.clear()
    hoja.update([dfrm.columns.values.tolist()] + dfrm.values.tolist())

#fill_sheets(hojas,df)

df

,DrDoeg,lil late,Paulina,Lau,Pamela,Seco,Joeq,jj
DrDoeg,None,None,None,None,None,None,None,None
lil late,None,None,None,None,None,None,None,None
Paulina,None,None,None,None,None,None,None,None
Lau,None,None,None,None,None,None,None,None
Pamela,None,None,None,None,None,None,None,None
Seco,None,None,None,None,None,None,None,None
Joeq,None,None,None,None,None,None,None,None
jj,None,None,None,None,None,None,None,None


# Mensajes Abiertos: Sheet 1 (Open Messages: Sheet 1)

***
**SHEET 1**: Este es un lugar para charlas públicas informales.\
 Todos los mensajes son visibles y cualquier participante puede escribir en cualquier celda. No hay seguridad.
***
**SHEET 1**: This is a place for casual public chats.\
All messages are visible and any participant can write to any cell. There is no security.
***
- la fila representa el remitente\
(the row represents the sender)
- la columna representa  el destinatario\
(column represents the recipient)

In [71]:
def sheet2df(she):
  datafr = pd.DataFrame(she.get_all_records())
  datafr.index=datafr.columns
  return datafr

In [72]:
dataframe1 = sheet2df(sheet1)
dataframe1

,DrDoeg,lil late,Paulina,Lau,Pamela,Seco,Joeq,jj
DrDoeg,None,None,None,None,None,None,None,None
lil late,None,None,None,None,None,None,None,None
Paulina,None,None,None,None,None,None,None,None
Lau,None,None,None,None,None,None,None,None
Pamela,None,None,None,None,None,None,None,None
Seco,None,None,None,None,None,None,None,None
Joeq,None,None,None,None,None,None,None,None
jj,None,None,None,hola,None,None,None,None


## Crear Widget para Seleccionar el Destinatario del Mensaje (Create Widget for Selecting Message Recipient) 

In [65]:
import ipywidgets as widgets
w=widgets.Dropdown(
    options=participants,
    value='DrDoeg',
    description='Recipient:',
    disabled=False,
)

In [66]:
#variable returns dropdown menu
#change value by clicking on it
w

Dropdown(description='Recipient:', options=('DrDoeg', 'lil late', 'Paulina', 'Lau', 'Pamela', 'Seco', 'Joeq', …

In [67]:
#
w.value

'DrDoeg'

## Crear u Publicar Mesaje (Create and Post Message)

- Seleccionar contenedor\
(Select recipient)
- Escribir un mensaje\
(Write message)
- Publicar mensaje\
(Publish message)
- Ver tablero de mensajes\
(View message board)

In [77]:
print(f'Vos eres:')
print(f'You are: {my_name}')
print('~'*20)
print('Selectar destinatario del mensaje')
print('Select a message recipient')
w

Vos eres:
You are: jj
~~~~~~~~~~~~~~~~~~~~
Selectar destinatario del mensaje
Select a message recipient


Dropdown(description='Recipient:', index=3, options=('DrDoeg', 'lil late', 'Paulina', 'Lau', 'Pamela', 'Seco',…

In [69]:
p_dict=dict([ (e,i) for i,e in enumerate(participants)])
message=input('send a message')
write_cell=(p_dict[my_name]+2,p_dict[w.value]+1)
sheet1.update_cell(*write_cell, message)

send a messagehola


{'spreadsheetId': '1z_iLwzlqa_JX7AdRR3OsBeFEgvS0jVYh1oo-X23bMOY',
 'updatedCells': 1,
 'updatedColumns': 1,
 'updatedRange': 'Magic!D9',
 'updatedRows': 1}

In [73]:
dataframe1 = sheet2df(sheet1)
dataframe1

,DrDoeg,lil late,Paulina,Lau,Pamela,Seco,Joeq,jj
DrDoeg,None,None,None,None,None,None,None,None
lil late,None,None,None,None,None,None,None,None
Paulina,None,None,None,None,None,None,None,None
Lau,None,None,None,None,None,None,None,None
Pamela,None,None,None,None,None,None,None,None
Seco,None,None,None,None,None,None,None,None
Joeq,None,None,None,None,None,None,None,None
jj,None,None,None,hola,None,None,None,None


In [78]:
dataframe1[my_name]

DrDoeg      None
lil late    None
Paulina     None
Lau         None
Pamela      None
Seco        None
Joeq        None
jj          None
Name: jj, dtype: object

# Mensajes Cifradas (Encrypted Messages) : Sheet2

In [79]:
dataframe2 = sheet2df(sheet2)
dataframe2

,DrDoeg,lil late,Paulina,Lau,Pamela,Seco,Joeq,jj
DrDoeg,None,None,None,None,None,None,None,None
lil late,None,None,None,None,None,None,None,None
Paulina,None,None,None,None,None,None,None,None
Lau,None,None,None,None,None,None,None,None
Pamela,None,None,None,None,None,None,None,None
Seco,None,None,None,None,None,None,None,None
Joeq,None,None,None,None,None,None,None,None
jj,None,None,None,None,None,None,None,None


In [80]:
def encrypt_message(hexPubKey,message):
    if type(message)!=bytes:
        message=message.encode()
    return ecies.encrypt(hexPubKey,message).hex()

In [81]:
w

Dropdown(description='Recipient:', index=3, options=('DrDoeg', 'lil late', 'Paulina', 'Lau', 'Pamela', 'Seco',…

In [87]:
message=input('send an encrypted message')
write_cell=(p_dict[my_name]+2,p_dict[w.value]+1)
sheet2.update_cell(*write_cell, encrypt_message(dataframe.set_index('name').pubkey[w.value],message))

send an encrypted messageme


{'spreadsheetId': '1z_iLwzlqa_JX7AdRR3OsBeFEgvS0jVYh1oo-X23bMOY',
 'updatedCells': 1,
 'updatedColumns': 1,
 'updatedRange': 'ChorusSignature!H9',
 'updatedRows': 1}

In [88]:
dataframe2 = sheet2df(sheet2)
dataframe2

,DrDoeg,lil late,Paulina,Lau,Pamela,Seco,Joeq,jj
DrDoeg,None,None,None,None,None,None,None,None
lil late,None,None,None,None,None,None,None,None
Paulina,None,None,None,None,None,None,None,None
Lau,None,None,None,None,None,None,None,None
Pamela,None,None,None,None,None,None,None,None
Seco,None,None,None,None,None,None,None,None
Joeq,None,None,None,None,None,None,None,None
jj,None,None,None,04da56c8c4854ac6ff84ef169ac8b7fcef9bec055d12c8...,None,None,None,04a026888092a8f67c77e8cf6626912f22159df6cb2236...


In [93]:
def import_privKey(path,password):
  import hashlib
  if type(password)!=bytes:
    password=password.encode()
  f=open(path,'rb')
  decrypted_bytes=ecies.aes_decrypt(key=hashlib.sha256(password).digest(),cipher_text=f.read())
  privKey=eth_keys.keys.PrivateKey(decrypted_bytes)
  f.close()
  return privKey

def decrypt_message(privKey,cipherhex):
    try:
      ciphertext=bytes.fromhex(cipherhex)
      return ecies.decrypt(privKey.to_hex(),ciphertext)
    except:
      return None

In [96]:
decrypt_message(import_privKey('llaves/prvkey1.eck',''),dataframe2[my_name][my_name])

b'me'

In [97]:
k=import_privKey('llaves/prvkey1.eck','')
tr=dataframe2[my_name].apply(lambda x: decrypt_message(k,x))
tr

DrDoeg       None
lil late     None
Paulina      None
Lau          None
Pamela       None
Seco         None
Joeq         None
jj          b'me'
Name: jj, dtype: object

# Firmada y Cifrada (Signed and Encrypted) : Sheet3

In [ ]:
def sign_message(privKey,message):
    if type(message)!=bytes:
        message=message.encode()
    signature= privKey.sign_msg(message)
    return signature.to_bytes().hex()

In [ ]:
mess=input('message')
mess_sig=sign_message(import_privKey('llaves/prvkey1.eck',''),mess)
mess_sig

In [ ]:
emess=encrypt_message(pubkey1,mess)
emess

In [ ]:
emess_sig=sign_message(k,bytes.fromhex(emess))
emess_sig,len(emess_sig)

In [ ]:
emess_sig,emess

In [ ]:
def sign_encrypt_message(privKey,hexpubKey,message):
    if type(message)!=bytes:
        message=message.encode()
    byte_enc=bytes.fromhex(encrypt_message(hexpubKey,message))
    signature= privKey.sign_msg(byte_enc)
    return signature.to_bytes().hex()+byte_enc.hex()

In [ ]:
sgen=sign_encrypt_message(k,pubkey1,mess)
sgen

In [ ]:
def decrypt_verify(pubkey,privkey,block):
  try:
    ver=verify(block[:130],bytes.fromhex(block[130:]),pubkey)
    return decrypt_message(privkey,block[130:]),ver
  except:
    return None

In [ ]:
decrypt_verify(pubkey1,k,sgen)

In [ ]:
message=input('send a signed and encrypted message')
write_cell=(p_dict[my_name]+2,p_dict[w.value]+1)
signed_enc_message=sign_encrypt_message(k,pubkey1,message)
sheet3.update_cell(*write_cell, encrypt_message(dataframe.set_index('name').pubkey[w.value],signed_enc_message))
dataframe3 = pd.DataFrame(sheet3.get_all_records())
dataframe3.index=dataframe3.columns
dataframe3

In [ ]:
dataframe3 = pd.DataFrame(sheet3.get_all_records())
dataframe3.index=dataframe3.columns
dataframe3

In [ ]:
pubkey_name=dataframe.set_index('name')['pubkey']
dataframe3[[my_name]].reset_index().apply(lambda x: decrypt_verify(pubkey_name[x['index']],k,x[my_name])  ,axis=1)